<a href="https://colab.research.google.com/github/iraidaantropova/machine_L/blob/main/HW9_SinitsaI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 9. Интеграция. Итоговый проект

Стандартная версия: Нужно реализовать rest api на базе flask в google colab.

1. выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813),
2. сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
3. Реализовать ноутбук с сервером
4. Реализовать ноутбук с клиентом


In [10]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.9 MB/s eta 0:00:00


In [14]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

from sklearn.metrics import f1_score
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [46]:
# датасет взяла : "https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction"
#загружаю данные

df = pd.read_csv("Heart Attack Data Set.csv")
df.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1


In [47]:
#age, trestbps, chol, thalach, oldpeak являются непрерывными признаками, а остальные-категориальными

#след шаг:  пайплайн для обработки признаков

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [48]:
from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
cat_cols = ['sex','cp', 'fbs', 'exang', 'slope']
base_cols = ['restecg', 'ca', 'thal']


continuos_transformers = []
cat_transformers = []
base_transformers=[]


for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))

for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))

for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [49]:
from sklearn.model_selection import train_test_split
#разделяю данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', 1),
                                                    df['target'], random_state=24)

<ipython-input-49-3e9218232e18>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_train, X_test, y_train, y_test = train_test_split(df.drop('target', 1),


In [50]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)


array([[-6.75941412e-01, -4.34565489e-01,  5.14025007e-01, ...,
         0.00000000e+00,  0.00000000e+00,  3.00000000e+00],
       [ 5.49785317e-01,  1.37629209e-01, -8.13865964e-01, ...,
         1.00000000e+00,  2.00000000e+00,  2.00000000e+00],
       [-1.23308993e+00,  2.31902697e-02,  1.83848970e-03, ...,
         0.00000000e+00,  4.00000000e+00,  3.00000000e+00],
       ...,
       [ 2.22123086e+00, -6.63443368e-01,  4.19175652e-01, ...,
         0.00000000e+00,  1.00000000e+00,  2.00000000e+00],
       [ 1.77551205e+00,  1.39645754e+00, -3.61012523e-02, ...,
         0.00000000e+00,  0.00000000e+00,  2.00000000e+00],
       [-7.36319621e-03, -6.63443368e-01, -1.11738390e+00, ...,
         1.00000000e+00,  1.00000000e+00,  3.00000000e+00]])

In [51]:
#обучение пайплайна
from sklearn.ensemble import RandomForestClassifier
cls = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 0)),
])
cls.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('trestbps',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='trestbps')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('chol',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='chol')),
                                                                 ('standard',
                                                                  StandardScaler(...
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelector(key='slope')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='slope'))])),
                                                ('restecg',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='restecg'))])),
                                                ('ca',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='ca'))])),
                                                ('thal',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='thal'))]))])),
                ('classifier', RandomForestClassifier(random_state=0))])

In [52]:
cls.steps

[('features',
  FeatureUnion(transformer_list=[('age',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='age')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('trestbps',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='trestbps')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('chol',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='chol')),
                                                  ('standard',
                                                   StandardScaler())])),
                 

In [53]:
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [54]:
#сохраняю модель:
with open("RandForcls.dill", "wb") as f:
    dill.dump(cls, f)

Проверка работоспособности и качества пайплайна

In [56]:
import pandas as pd
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

import dill
dill._dill._reverse_typemap['ClassType'] = type

In [57]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [58]:
X_test.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,58,1,0,150,270,0,0,111,1,0.8,2,0,3
1,60,1,0,130,253,0,1,144,1,1.4,2,1,3
2,45,0,1,112,160,0,1,138,0,0.0,1,0,2


In [59]:
with open('RandForcls.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [60]:
predictions = pipeline.predict_proba(X_test.iloc[:500])
pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

In [61]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test.iloc[:500])

0.9094076655052266